In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction import text
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score

In [24]:
fiction_df = pd.read_csv('./data/fiction_sample.csv')

In [ ]:
# models to evaluate
- Logistic Regression
Random Forest
Support Vector
Neural Network
Naive Bayes

### Functions

In [25]:
def best_params(pipeline, params, X_train, y_train):
    gs = GridSearchCV(pipeline,
                      param_grid = params,
                      n_jobs=-1)

    gs.fit(X_train, y_train)
    return f'Best Score: {gs.best_score_}, Params: {gs.best_params_}'

In [26]:
def return_gs(pipeline, params, X_train, y_train):
    gs = GridSearchCV(pipeline,
                      param_grid = params,
                      n_jobs=-1)
    return gs

In [27]:
def scores(gs, X_train, y_train, X_test, y_test):
    gs.fit(X_train, y_train)
    return f'Train Score: {gs.score(X_train, y_train)}, Test Score: {gs.score(X_test, y_test)}'

In [29]:
def predictions(pipeline, X_train, X_test, y_train):
    pipeline.fit(X_train, y_train)
    prediction = pipeline.predict(X_test)
    
    return prediction

In [28]:
def classification_scores(model, y_test, y_pred):
    dataframe = pd.DataFrame(columns = ['Recall', 'Precision', 'F1', 'Accuracy'])
    
    recall = recall_score(y_test, y_pred, average = 'weighted')
    precision = precision_score(y_test, y_pred, average = 'weighted')
    f1 = f1_score(y_test, y_pred, average = 'weighted')
    accuracy = accuracy_score(y_test, y_pred)
    
    dataframe.loc[model] = [recall, precision, f1, accuracy]
    
    return dataframe

## Fiction Genre

### Baseline Accuracy 

In [30]:
fiction_df['Title'].value_counts(normalize = True)

Pride and Prejudice                                                 0.02836
Great Expectations                                                  0.00840
Brave New World                                                     0.00740
The Great Gatsby                                                    0.00678
The Hobbitt, or there and back again; illustrated by the author.    0.00592
                                                                     ...   
Man's Fate                                                          0.00002
That Summer                                                         0.00002
Heller with a Gun                                                   0.00002
The Stalkers (Plainsmen (Audio))                                    0.00002
Star Trek 2005 Wall Calendar                                        0.00002
Name: Title, Length: 9379, dtype: float64

In [36]:
X = fiction_df['description']
y = fiction_df['Title']

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [38]:
def my_lemmatizer(text):
    wnet = WordNetLemmatizer()
    # exclude words with apostrophes and numbers
    return [wnet.lemmatize(w) for w in text.split() if "'" not in w and not w.isdigit()]

In [39]:
wnet = WordNetLemmatizer()
lem_stopwords = [wnet.lemmatize(w) for w in stopwords.words('english')]

contractions = ['ve', 't', "'s'", 'd', 'll', 'm', 're']
lem_contractions = [wnet.lemmatize(contraction) for contraction in contractions]

numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
lem_numbers = [wnet.lemmatize(num) for num in numbers]

lem_stopwords = lem_stopwords + lem_contractions + lem_numbers

### MultinomialNB

In [40]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# models to evaluate
Support Vector
Neural Network

In [42]:
mnb_pipe = Pipeline([
    ('tf', TfidfVectorizer(stop_words = lem_stopwords, 
                           tokenizer = my_lemmatizer,
                           token_pattern = None,
                           max_features = 5_000)),
    ('mnb', MultinomialNB(alpha = 0.5))
])

In [43]:
mnb_params = {
    'tf__min_df': [0.1, 0.25, 0.5, 1.0],
    'tf__max_df': [0.25, 0.5, 0.8, 1.0],
    'tf__ngram_range': [(1,1), (2,2), (3,3)],
    'mnb__alpha': [0.1, 0.25, 0.5, 1],
    'mnb__fit_prior': [True, False]
}

In [44]:
best_params(mnb_pipe, mnb_params, X_train, y_train)

/Users/lisaliang/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/lisaliang/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
1320 fits failed out of a total of 1920.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
840 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/lisaliang/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/lisaliang/opt/anaconda3/lib/python3.9/site-packa

"Best Score: 0.6556000000000001, Params: {'mnb__alpha': 0.1, 'mnb__fit_prior': False, 'tf__max_df': 0.5, 'tf__min_df': 0.1, 'tf__ngram_range': (1, 1)}"

In [45]:
mnb_gs = return_gs(mnb_pipe, mnb_params, X_train, y_train)

In [46]:
scores(mnb_gs, X_train, y_train, X_test, y_test)

/Users/lisaliang/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/lisaliang/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
1320 fits failed out of a total of 1920.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
840 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/lisaliang/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/lisaliang/opt/anaconda3/lib/python3.9/site-packa

'Train Score: 0.7156, Test Score: 0.6444'

In [47]:
mnb_pred = predictions(mnb_pipe, X_train, X_test, y_train)

In [48]:
classification_scores('Multinomial Naive Bayes', y_test, mnb_pred)

/Users/lisaliang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/lisaliang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Recall,Precision,F1,Accuracy
Multinomial Naive Bayes,0.63488,0.490088,0.535894,0.63488


In [ ]:
with open('fiction_pipe.pkl', 'wb') as f:
    pickle.dump(pipe, f)

### Random Forest Classification

In [56]:
rfc_pipe = Pipeline([
    ('tf', TfidfVectorizer(stop_words = lem_stopwords, 
                           tokenizer = my_lemmatizer,
                           token_pattern = None,
                           max_features = 5_000)),
    ('rfc', RandomForestClassifier())
])

In [57]:
rfc_params = {
    'tf__min_df': [0.1, 0.25, 0.5, 1.0],
    'tf__max_df': [0.25, 0.5, 0.8, 1.0],
    'tf__ngram_range': [(1,1), (2,2), (3,3)],
    'rfc__n_estimators': [100, 200, 300],
    'rfc__max_depth': [5, 10, 20],
    'rfc__bootstrap': [True, False]
}

In [ ]:
best_params(rfc_pipe, rfc_params, X_train, y_train)

/Users/lisaliang/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [ ]:
rfc_gs = return_gs(rfc_pipe, rfc_params, X_train, y_train)

In [ ]:
scores(rfc_gs, X_train, y_train, X_test, y_test)

In [ ]:
rfc_pred = predictions(rfc_pipe, X_train, X_test, y_train)

In [ ]:
classification_scores('Random Forest Classifier', y_test, rfc_pred)

### Logistic Regression

In [ ]:
lr_pipe = Pipeline([
    ('tf', TfidfVectorizer(stop_words = lem_stopwords, 
                           tokenizer = my_lemmatizer,
                           token_pattern = None,
                           max_features = 5_000)),
    ('rfc', RandomForestClassifier(max_features = 5_000))
])

In [ ]:
lr_params = {
    'tf__min_df': [0.1, 0.25, 0.5, 1.0],
    'tf__max_df': [0.25, 0.5, 0.8, 1.0],
    'tf__ngram_range': [(1,1), (2,2), (3,3)],
    'rfc__n_estimators': [100, 200, 300],
    'rfr__max_depth': [5, 10, 20],
    'rfc__bootstrap': [True, False]
}

In [ ]:
best_params(lr_pipe, lr_params, X_train, y_train)

In [ ]:
lr_gs = return_gs(lr_pipe, lr_params, X_train, y_train)

In [ ]:
scores(lr_gs, X_train, y_train, X_test, y_test)

In [ ]:
lr_pred = predictions(lr_pipe, X_train, X_test, y_train)

In [ ]:
classification_scores('Logistic Regression', y_test, lr_pred)

### Support Vector